In [4]:
!pip install transformers -q

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-enron-spam-detection")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-tiny-finetuned-enron-spam-detection")

In [16]:
import pandas as pd

message = '''buy online and save viagra price for this high demand med best price for this high demand med best price for this high demand med buy nowbuy nowbuy price for this high demand med best price for this high demand med best price for this high demand med buy nowbuy nowbuy nowcialis soft price for this high demand med best price for this high demand med best price for this high demand med buy nowbuy nowbuy your penis width ( girth ) by 20 % gain up to 3 + full inches in length buy nowbuy now'''

In [22]:
input = tokenizer(message, return_tensors='pt', padding=True)
input

{'input_ids': tensor([[  101,  4965,  3784,  1998,  3828,  3081, 17643,  3976,  2005,  2023,
          2152,  5157, 19960,  2190,  3976,  2005,  2023,  2152,  5157, 19960,
          2190,  3976,  2005,  2023,  2152,  5157, 19960,  4965,  2085,  8569,
          2100,  2085,  8569,  2100,  3976,  2005,  2023,  2152,  5157, 19960,
          2190,  3976,  2005,  2023,  2152,  5157, 19960,  2190,  3976,  2005,
          2023,  2152,  5157, 19960,  4965,  2085,  8569,  2100,  2085,  8569,
          2100,  2085, 13247,  2483,  3730,  3976,  2005,  2023,  2152,  5157,
         19960,  2190,  3976,  2005,  2023,  2152,  5157, 19960,  2190,  3976,
          2005,  2023,  2152,  5157, 19960,  4965,  2085,  8569,  2100,  2085,
          8569,  2100,  2115, 19085,  9381,  1006, 21025, 15265,  1007,  2011,
          2322,  1003,  5114,  2039,  2000,  1017,  1009,  2440,  5282,  1999,
          3091,  4965,  2085,  8569,  2100,  2085,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
output = model(**input)
logits = output.logits

In [19]:
import torch

prob = torch.nn.functional.softmax(logits, dim=-1)
predicted_class = torch.argmax(prob, dim=-1)
predicted_class = predicted_class.item()
print(predicted_class)

1


In [20]:
class_dict = {0: 'not_spam', 1: 'spam'}
predicted_class_label = class_dict[predicted_class]

In [21]:
print(predicted_class_label)

spam


In [23]:
spam_email_strings = [
    "Congratulations! You've won a luxury vacation!\nHi there,\n...",
    "Urgent: Your account needs immediate attention\nDear Customer,\n...",
    "Exclusive Limited Time Offer: 90% off all products!\nHello Shoppers,\n...",
    "You've won the lottery! Claim your millions now!\nCongratulations Winner,\n...",
    "Boost your business with our guaranteed marketing strategies\nDear Business Owner,\n...",
    "Important Tax Refund Notification\nDear Taxpayer,\n...",
    "Hot Singles in Your Area are Waiting to Chat!\nHey there,\n...",
    "Re: Your Invoice #12345\nDear Valued Customer,\n...",
    "Exclusive Investment Opportunity: Guaranteed 300% ROI\nHello Investor,\n...",
    "Claim Your Inheritance from a Distant Relative\nDearest Beneficiary,\n..."
]

data = {'Email': spam_email_strings}
df = pd.DataFrame(data)

df.to_csv('spam_dataset.csv', index=False)

In [24]:
df2 = pd.read_csv('spam_dataset.csv')
df2

,Email
0,Congratulations! You've won a luxury vacation!...
1,Urgent: Your account needs immediate attention...
2,Exclusive Limited Time Offer: 90% off all prod...
3,You've won the lottery! Claim your millions no...
4,Boost your business with our guaranteed market...
5,Important Tax Refund Notification\nDear Taxpay...
6,Hot Singles in Your Area are Waiting to Chat!\...
7,"Re: Your Invoice #12345\nDear Valued Customer,..."
8,Exclusive Investment Opportunity: Guaranteed 3...
9,Claim Your Inheritance from a Distant Relative...


In [27]:
def evaluate_transformer(message):
  input = tokenizer(message, return_tensors='pt', padding=True)
  output = model(**input)
  logits = output.logits
  prob = torch.nn.functional.softmax(logits, dim=-1)
  predicted_class = torch.argmax(prob, dim=-1)
  predicted_class = predicted_class.item()
  class_dict = {0: 'not_spam', 1: 'spam'}
  predicted_class_label = class_dict[predicted_class]
  return predicted_class_label

In [28]:
messages = df2["Email"].unique()
evaluations = []
for m in messages:
  evaluations.append(evaluate_transformer(m))

In [29]:
df_clean = pd.DataFrame({'Messages': messages, 'Type': evaluations})
df_clean

,Messages,Type
0,Congratulations! You've won a luxury vacation!...,spam
1,Urgent: Your account needs immediate attention...,spam
2,Exclusive Limited Time Offer: 90% off all prod...,spam
3,You've won the lottery! Claim your millions no...,spam
4,Boost your business with our guaranteed market...,spam
5,Important Tax Refund Notification\nDear Taxpay...,spam
6,Hot Singles in Your Area are Waiting to Chat!\...,spam
7,"Re: Your Invoice #12345\nDear Valued Customer,...",spam
8,Exclusive Investment Opportunity: Guaranteed 3...,spam
9,Claim Your Inheritance from a Distant Relative...,spam
